In [1]:
import os

import pandas as pd

In [2]:
pd.options.display.max_columns = None

In [3]:
kbr_data_path = '../../data/raw/KBR/'
kbr_dir_path = '1_Ankiety_w_gospodarstwach_domowych/'
kbr_file_path = 'Etap_V_1_1_Ankiety_w_gospodarstwach_domowych_Wroclaw.xlsx'
excel_sheet = 'Wrocław_ankiety+podróże'

data = pd.read_excel(os.path.join(kbr_data_path, kbr_dir_path, kbr_file_path), excel_sheet, header=[0,1], engine='openpyxl')

data.columns = [f'{i}_{j}' for i, j in data.columns]
data = data.iloc[0:14241]

if data.shape[1] == len(set(list(data.columns))):
    print('All column names are unique.')
else:
    print('All column names are NOT unique.')

All column names are unique.


In [4]:
data.head()

,Lp_Lp,IDENTYFIKACJA ANKIETY_ID_LOS (numer kwestionariusza),IDENTYFIKACJA ANKIETY_Nr ankietera,IDENTYFIKACJA ANKIETY_Data wypełnienia kwestionariusza,IDENTYFIKACJA ANKIETY_Godzina wypełnienia kwestionariusza,IDENTYFIKACJA ANKIETY_Nr rejonu komunikacyjnego,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Wprowadzenie opłat za wjazd do centrum dla wszystkich pojazdów,"PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Dopuszczenie dojazdu do centrum dla pojazdów, które nie zanieczyszczają środowiska (np. pojazdy elektryczne, hybrydowe lub spełniające najwyższe normy spalania)","PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jaka jest Pana/Pani opinia na temat rozwiązań wspierających kursowanie tramwajów i autobusów, takich jak priorytet przejazdu na skrzyżowaniach oraz wydzielone pasy na jezdniach?",PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Z jakiego środka transportu zbiorowego korzysta Pan/Pani najczęściej?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Dlaczego? \n[tramwaj/autobus],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Dlaczego? [Inne]\n[tramwaj/autobus],"PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jaka jest Pana/Pani opinia na temat rozwoju systemu kolei aglomeracyjnej na terenie Wrocławia i wokół niego (rozbudowa układu torowego, zwiększenie częstotliwości itd.)?",PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jak Pan/Pani ocenia wygodę jazdy pojazdami komunikacji zbiorowej?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jak Pan/Pani ocenia czystość taboru komunikacji zbiorowej we Wrocławiu?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jak Pan/Pani ocenia punktualność komunikacji zbiorowej we Wrocławiu?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jak ocenia Pan/Pani efekty dotychczasowych działań związanych z rozbudową systemu rowerowego we Wrocławiu (dróg i parkingów rowerowych)?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [niekorzystne ustawienia sygnalizacji świetlnej],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [brak chodników i konieczność poruszania się jezdnią/poboczem/wydeptaną ścieżką],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zły stan nawierzchni chodników],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zastawianie chodników przez parkujące samochody],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [niebezpieczne zachowania kierowców],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zagrożenie ze strony rowerzystów poruszających się chodnikami],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zbyt wysokie krawężniki],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [brak bieżącego utrzymania czystości/odśnieżania],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [niewystarczająca liczba przejść dla pieszych],"PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wska

In [5]:
chosen_cols = [
    'IDENTYFIKACJA ANKIETY_ID_LOS (numer kwestionariusza)',
    'DANE O RESPONDENCIE_Przedział wiekowy',
    'DANE O RESPONDENCIE_Płeć',
    'OPIS PODRÓŻY "ŹRÓDŁO"_Z jakiego miejsca',
    'OPIS PODRÓŻY "CEL"_Do jakiego miejsca'
]

data_filtered = data[chosen_cols]

data_filtered.columns = [
    'id', 
    'age_bracket', 
    'sex', 
    'source', 
    'destination'
]

data_filtered.fillna(0, inplace=True)
# data_filtered.dropna(inplace=True)
# data_filtered.reset_index(drop=True, inplace=True)

In [6]:
age_bracket_map_dict = {
    '6-15 (dzieci)': '6-15',
    '16-19 (młodzież)': '16-19',
    '20-24 (wiek studencki)': '20-24',
    '25-44 (młodsi pracownicy)': '25-44',
    '45-60 (starsi pracownicy kobiety)': '45-60',  # '45-60(K)',
    '45-65 (starsi pracownicy mężczyźni)': '45-65',  # '45-65(M)',
    '61 i więcej (emeryci kobiety)': '61-x',  # '61-(K)',
    '66 i więcej (emeryci mężczyźni)': '66-x'  # '66-(M)'
}
data_filtered.replace({'age_bracket': age_bracket_map_dict}, inplace=True)

sex_map_dict = {
    'Kobieta': 'K',
    'Mężczyzna': 'M'
}
data_filtered.replace({'sex': sex_map_dict}, inplace=True)

data_filtered['age_sex_comb'] = data_filtered['age_bracket'] + '_' + data_filtered['sex']

In [7]:
data_filtered['source'] = data_filtered['source'].str.replace('Inne', 'inne')
data_filtered['destination'] = data_filtered['destination'].str.replace('Inne', 'inne')

In [8]:
count_series = data_filtered.groupby(['age_sex_comb', 'destination']).size()
destination_df = count_series.to_frame(name = 'count').reset_index()

In [9]:
destination_df = destination_df.pivot(index='age_sex_comb', columns='destination')['count']
destination_df = destination_df.fillna(0)

In [10]:
destination_df

destination,dom,inne,praca,szkoła,uczelnia
age_sex_comb,,,,,
16-19_K,88.0,28.0,7.0,59.0,12.0
16-19_M,129.0,36.0,5.0,93.0,12.0
20-24_K,280.0,143.0,57.0,12.0,140.0
20-24_M,259.0,113.0,55.0,8.0,133.0
25-44_K,1023.0,441.0,643.0,81.0,24.0
25-44_M,941.0,292.0,732.0,21.0,26.0
45-60_K,654.0,359.0,386.0,15.0,0.0
45-65_M,750.0,325.0,516.0,10.0,2.0
6-15_K,321.0,61.0,1.0,281.0,0.0


In [11]:
destination_df['sum'] = destination_df['dom'] + destination_df['inne'] + destination_df['praca'] + destination_df['szkoła'] + destination_df['uczelnia']

destination_df['dom'] = destination_df['dom'] / destination_df['sum']
destination_df['inne'] = destination_df['inne'] / destination_df['sum']
destination_df['praca'] = destination_df['praca'] / destination_df['sum']
destination_df['szkoła'] = destination_df['szkoła'] / destination_df['sum']
destination_df['uczelnia'] = destination_df['uczelnia'] / destination_df['sum']

destination_df.drop(columns=['sum'], inplace=True)

In [12]:
destination_df.style.format('{:.3f}')

destination,dom,inne,praca,szkoła,uczelnia
age_sex_comb,,,,,
16-19_K,0.454,0.144,0.036,0.304,0.062
16-19_M,0.469,0.131,0.018,0.338,0.044
20-24_K,0.443,0.226,0.090,0.019,0.222
20-24_M,0.456,0.199,0.097,0.014,0.234
25-44_K,0.462,0.199,0.291,0.037,0.011
25-44_M,0.468,0.145,0.364,0.010,0.013
45-60_K,0.463,0.254,0.273,0.011,0.000
45-65_M,0.468,0.203,0.322,0.006,0.001
6-15_K,0.483,0.092,0.002,0.423,0.000
